In [1]:
import time

notebook_start_time = time.time()

# Set up environment

In [2]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

⛳️ Local environment
Adding the following directory to the PYTHONPATH: /Users/pauliusztin/Documents/01_projects/hopsworks_recsys/hands-on-recommender-system


# Inference pipeline: Deploying and testing the inference pipeline 

In this notebook, we will dig into the inference pipeline and deploy it to Hopsworks as a real-time service.

## 📝 Imports

In [3]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [4]:
project, fs = hopsworks_integration.get_feature_store()

2024-12-21 10:58:23.565 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:13 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2024-12-21 10:58:23,566 INFO: Initializing external client
2024-12-21 10:58:23,566 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-21 10:58:25,057 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192098


# Deploying the ranking inference pipeline


You start by deploying your ranking model. Since it is a CatBoost model you need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it:

In [5]:
ranking_deployment = hopsworks_integration.ranking_serving.HopsworksRankingModel.deploy(
    project=project
)

Uploading: 100.000%|██████████| 4491/4491 elapsed<00:01 remaining<00:00
Uploading: 100.000%|██████████| 1113/1113 elapsed<00:02 remaining<00:00


Deployment created, explore it at https://c.app.hopsworks.ai:443/p/1192098/deployments/352268
Before making predictions, start the deployment by using `.start()`


Now, we have to explicitly start the deployment:

In [6]:
ranking_deployment.start()

Deployment is ready: 100%|██████████| 6/6 [00:26<00:00,  4.41s/it]    

Start making predictions by using `.predict()`


In [7]:
# Check logs in case of failure
# ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/1192098/deployments/352268

DeployableComponentLogs(instance_name: 'ranking-transformer-00001-deployment-74f888dcb6-pmgxz', date: datetime.datetime(2024, 12, 21, 10, 59, 20, 47920)) 
INFO:root:Loading component module...
INFO:root:[TransformerModel] Initializing transformer for model: ranking
INFO:root:[HopsworksModel] Initializing for model: ranking
INFO:hsfs.engine.python:Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai/p/1192098
... execution time: 10.796460 seconds
INFO:root:Starting KServe server...
2024-12-21 08:59:16.742 8 kserve INFO [model_server.py:register_model():363] Registering model: ranking
2024-12-21 08:59:16.742 8 kserve INFO [model_server.py:start():298] Setting max asyncio worker threads as 12
2024-12-21 08:59:16.742 8 kserve INFO [model_server.py:_serve_rest():244] Starting uvicorn with 1 workers


2024-12-21 08:59:16.778 uvicorn.error

## <span style="color:#ff5f27"> Test the ranking inference pipeline</span>


In [8]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

Let's define a dummy test example to test our ranking deployment (only the `customer_id` has to match):

In [20]:
test_ranking_input = [
        {
            "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
            "month_sin": 1.2246467991473532e-16,
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ],
            "month_cos": -1.0,
        }
    ]

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(inputs=test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

Check logs in case of failure:

In [21]:
# ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/1192098/deployments/352268



RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1192098/serving/352268/logs). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":240027,"errorMsg":"Server logs not available"}', error code: 240027, error msg: Server logs not available, user msg: 

# Deploying the query inference pipeline

In [11]:
query_model_deployment = (
    hopsworks_integration.two_tower_serving.HopsworksQueryModel.deploy()
)

Uploading: 100.000%|██████████| 2501/2501 elapsed<00:02 remaining<00:00


Deployment created, explore it at https://c.app.hopsworks.ai:443/p/1192098/deployments/352269
Before making predictions, start the deployment by using `.start()`


At this point, you have registered your deployment. To start it up you need to run:

In [15]:
query_model_deployment.start()

Deployment is already running


In [16]:
# Check logs in case of failure
# query_model_deployment.get_logs(component="transformer", tail=20)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/1192098/deployments/352269



RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1192098/serving/352269/logs). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":240027,"errorMsg":"Server logs not available"}', error code: 240027, error msg: Server logs not available, user msg: 

## <span style="color:#ff5f27"> Testing the inference pipeline </span>

Define a test input example:

In [17]:
data = [
    {
        "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
        "transaction_date": "2022-11-15T12:16:25.330916",
    }
]

Test out the deployment:

In [18]:
ranked_candidates = query_model_deployment.predict(inputs=data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

RestAPIError: Metadata operation error: (url: http://15.235.46.163/v1/models/query:predict). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, body: b'{"error":"HTTPError : HTTP 500: HTTPConnectionPool(host=\'10.1.2.158\', port=32080): Max retries exceeded with url: /v1/models/ranking:predict (Caused by NewConnectionError(\'<urllib3.connection.HTTPConnection object at 0x7fcc0af9e8f0>: Failed to establish a new connection: [Errno 113] No route to host\'))"}', error code: , error msg: , user msg: 

 Check the model server logs by using `.get_logs()`

Check logs in case of failure:

In [16]:
# query_model_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/1192098/deployments/352267

DeployableComponentLogs(instance_name: 'query-transformer-00001-deployment-8568b759fd-8pxvw', date: datetime.datetime(2024, 12, 21, 10, 9, 15, 530875)) 
    return self._serving_engine.predict(self, data, inputs)
  File "/srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/hsml/engine/serving_engine.py", line 580, in predict
    return self._serving_api.send_inference_request(
  File "/srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/hsml/core/serving_api.py", line 233, in send_inference_request
    return self._send_inference_request_via_rest_protocol(
  File "/srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-packages/hsml/core/serving_api.py", line 277, in _send_inference_request_via_rest_protocol
    return _client._send_request(
  File "/srv/hops/anaconda/envs/hopsworks_environment/lib/python3.10/site-pack

# <span style="color:#ff5f27"> Stopping the Hopsworks deployments </span>

Stop the deployment when you're not using it.

In [ ]:
ranking_deployment.stop()
query_model_deployment.stop()

## <span style="color:#ff5f27"> Inspecting the deployments in Hopsworks UI </span>

View results in [Hopsworks Serverless](https://rebrand.ly/serverless-github): **Data Science → Deployments**

---

In [ ]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)

# <span style="color:#ff5f27">→ Next Steps </span>

The last step is to schedule the materialization jobs.